In [15]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [16]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [17]:
columns = [
    "id", "Year", "Month", "InvDate",
    "InvNumber", "Market", "InvCustomer", "CompanyName",
    "ItemClass", "SubCategory", "Flavors", "Product",
    "Description", "Config", "UOM", "Real_Qty",
    "Cad_Value"]

target = ["Cad_Value"]

In [18]:
# Load the data
df = pd.read_csv('Sales_DataUse.csv')
#Data Before
df.head()

,YEAR,MONTH,InvDate,InvNumber,Market,InvCustomer,CompanyName,ItemClass,SubCategory,Flavours,Product,Description,Config,UOM,Real_Qty,CAD_Value
0,2013,7,7/4/2013,NA0223010005,CAN,CALL01,Callace & Warey,ORG,F&N,CASHEW,600065,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,CS,1,153.60
1,2013,7,7/4/2013,NA0223010003,CAN,SWAL01,Swalbo,ORG,F&N,CASHEW,600065,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,CS,22,2731.52
2,2013,7,7/4/2013,NA0223010004,CAN,CALL01,Callace & Warey,ORG,F&N,CASHEW,600065,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,CS,3,460.80
3,2013,7,7/4/2013,NA0223010005,CAN,CALL01,Callace & Warey,ORG,F&N,APPLE,600071,NAT CAN Niagara Apple Country,Organic 40gx16x8,CS,2,307.20
4,2013,7,7/4/2013,NA0223010004,CAN,CALL01,Callace & Warey,ORG,F&N,APPLE,600071,NAT CAN Niagara Apple Country,Organic 40gx16x8,CS,4,614.40


In [19]:
#Data After(no columns have been split out yet)
#df won't convert "Flavors"
df_encoded = pd.get_dummies(df, columns=["CompanyName", "ItemClass", "SubCategory", "Product", "UOM"])
df_encoded.head()

,YEAR,MONTH,InvDate,InvNumber,Market,InvCustomer,Flavours,Description,Config,Real_Qty,...,Product_601299,Product_601301,Product_601302,Product_601304,Product_601305,Product_601306,Product_601307,UOM_BAG,UOM_CAR,UOM_CS
0,2013,7,7/4/2013,NA0223010005,CAN,CALL01,CASHEW,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,1,...,0,0,0,0,0,0,0,0,0,1
1,2013,7,7/4/2013,NA0223010003,CAN,SWAL01,CASHEW,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,22,...,0,0,0,0,0,0,0,0,0,1
2,2013,7,7/4/2013,NA0223010004,CAN,CALL01,CASHEW,NAT CAN Cashew Nut Fiesta,Organic 40gx16x8,3,...,0,0,0,0,0,0,0,0,0,1
3,2013,7,7/4/2013,NA0223010005,CAN,CALL01,APPLE,NAT CAN Niagara Apple Country,Organic 40gx16x8,2,...,0,0,0,0,0,0,0,0,0,1
4,2013,7,7/4/2013,NA0223010004,CAN,CALL01,APPLE,NAT CAN Niagara Apple Country,Organic 40gx16x8,4,...,0,0,0,0,0,0,0,0,0,1


In [20]:
##Logistic Regression to Predict Sales

In [21]:
# Create our features
X = df_encoded.copy()
X = X.drop("CAD_Value", axis=1)
X = pd.get_dummies(X)
X.head()

,YEAR,MONTH,Real_Qty,CompanyName_ADS,CompanyName_Ab Scand,CompanyName_Accad Stores,CompanyName_Adfong,CompanyName_Amazing.ca,CompanyName_Amazing.com,CompanyName_Amazing.com central,...,Config_Organic 40gx16x3,Config_Organic 40gx16x56,Config_Organic 40gx16x8,Config_Organic 40gx18x1,Config_Organic 40gx18x10,Config_Organic 40gx18x15,Config_Organic 40gx18x18,Config_Organic 40gx18x2,Config_Organic 40gx3x10,Config_Organic 45gx14x18
0,2013,7,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2013,7,22,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2013,7,3,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2013,7,2,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2013,7,4,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [27]:
# Create our target
y = df_encoded["CAD_Value"]
y.head()

0     153.60
1    2731.52
2     460.80
3     307.20
4     614.40
Name: CAD_Value, dtype: float64

In [23]:
X.describe()

,YEAR,MONTH,Real_Qty,CompanyName_ADS,CompanyName_Ab Scand,CompanyName_Accad Stores,CompanyName_Adfong,CompanyName_Amazing.ca,CompanyName_Amazing.com,CompanyName_Amazing.com central,...,Config_Organic 40gx16x3,Config_Organic 40gx16x56,Config_Organic 40gx16x8,Config_Organic 40gx18x1,Config_Organic 40gx18x10,Config_Organic 40gx18x15,Config_Organic 40gx18x18,Config_Organic 40gx18x2,Config_Organic 40gx3x10,Config_Organic 45gx14x18
count,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,...,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000,35948.000000
mean,2016.888116,6.528847,60.065789,0.000862,0.022811,0.003839,0.003338,0.005007,0.001669,0.003310,...,0.000389,0.000195,0.535190,0.000028,0.000111,0.020585,0.017859,0.000028,0.009736,0.000056
std,2.500586,3.298315,327.341319,0.029354,0.149302,0.061841,0.057681,0.070585,0.040821,0.057441,...,0.019731,0.013953,0.498767,0.005274,0.010548,0.141993,0.132441,0.005274,0.098192,0.007459
min,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2017.000000,6.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.000000,9.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2021.000000,12.000000,11520.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# Check the balance of our target values
y.value_counts()

12.800       3127
0.000        1725
9.100        1526
134.090      1173
268.180      1132
             ... 
272.376         1
12314.654       1
15537.688       1
800.040         1
5367.648        1
Name: CAD_Value, Length: 2889, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [31]:
classifier.fit(X_train, y_train)

NameError: name 'X_train' is not defined